In [1]:
import hub
from PIL import Image
import numpy as np
import os, time
import torch #with fancy cuda
from torchvision import transforms, datasets, models
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import vgg16, VGG16_Weights
import matplotlib.pyplot as plt
import torch.optim as optim

In [2]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
                                transforms.Resize((400, 400)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(15),
                                transforms.ColorJitter(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])
test_transform = transforms.Compose([
                                transforms.Resize((400, 400)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])

train_data = datasets.ImageFolder('./car_data/car_data/train', train_transform)
test_data = datasets.ImageFolder('./car_data/car_data/test', test_transform)

In [3]:
classes = train_data.classes
class_idx = train_data.class_to_idx
len(class_idx)

196

In [4]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=2)

test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=2)

print(len(train_loader))

255


In [5]:
images, labels = next(iter(train_loader))
images.shape, len(labels)

(torch.Size([32, 3, 400, 400]), 32)

In [6]:
model = models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 196)

C:\Users\Anthony\anaconda3\envs\ecs171\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Anthony\anaconda3\envs\ecs171\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
for param in model.parameters():
    param.required_grad = False
print(model.classifier)

classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=196),
    nn.LogSoftmax(dim=1)
)
model.classifier = classifier
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=196, bias=True)
  (7): LogSoftmax(dim=1)
)

In [10]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
for param in model.parameters():
    param.required_grad = False

classifier = nn.Sequential(
  nn.Linear(in_features=2048, out_features=1024),
  nn.LeakyReLU(),
  nn.Dropout(p=0.2),
  nn.Linear(in_features=1024, out_features=512),
  nn.LeakyReLU(),
  nn.Dropout(p=0.3),
  nn.Linear(in_features=512, out_features=196),
  nn.LogSoftmax(dim=1)  
)
    
model.fc = classifier
model.fc

Sequential(
  (0): Linear(in_features=2048, out_features=1024, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=512, out_features=196, bias=True)
  (7): LogSoftmax(dim=1)
)

In [7]:
learning_rate = 0.001

# Loss Function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
def train_model(model, criterion, optimizer, scheduler, n_epochs = 5):
    
    # set the model to train mode initially
    model.train()
    for epoch in range(n_epochs):
        begin = time.time()
        running_loss = 0.0
        correct = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            correct += (labels==predicted).sum().item()

        epoch_duration = time.time()-begin
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100*correct/len(train_data)
        print("Epoch %s: time: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
        
        test_acc = eval_model(model)
        
        model.train()
        scheduler.step(test_acc)
        since = time.time()
    print('Finished Training')

In [9]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            #images = images.to(device).half() # uncomment for half precision model
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    print('Accuracy of the network on the test images: %d %%' % (
        test_acc))
    return test_acc

In [10]:
train_model(model, loss_fn, optimizer, lrscheduler, n_epochs=30)

Epoch 1: time: 371 s, loss: 5.2730, acc: 0.6508
Accuracy of the network on the test images: 1 %
Epoch 2: time: 396 s, loss: 5.0259, acc: 1.9155
Accuracy of the network on the test images: 5 %
Epoch 3: time: 413 s, loss: 4.2758, acc: 8.1041
Accuracy of the network on the test images: 17 %
Epoch 4: time: 401 s, loss: 3.2554, acc: 21.9180
Accuracy of the network on the test images: 35 %
Epoch 5: time: 1190 s, loss: 2.3458, acc: 39.1699
Accuracy of the network on the test images: 49 %
Epoch 6: time: 396 s, loss: 1.7748, acc: 52.0997
Accuracy of the network on the test images: 56 %
Epoch 7: time: 2322 s, loss: 1.3238, acc: 63.8139
Accuracy of the network on the test images: 62 %
Epoch 8: time: 384 s, loss: 0.9687, acc: 72.3477
Accuracy of the network on the test images: 65 %
Epoch 9: time: 394 s, loss: 0.4803, acc: 86.0388
Accuracy of the network on the test images: 74 %
Epoch 10: time: 2197 s, loss: 0.3542, acc: 89.4032
Accuracy of the network on the test images: 75 %
Epoch 11: time: 3528 

KeyboardInterrupt: 

In [12]:
torch.save(model.state_dict(), "./best_model.pt")

In [15]:
torch.save({'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict()}, 'model.pt')